In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys,argparse
import glob
import os
import cv2 
#can be installed by running "!pip install opencv-python"
#in current .ipynb
import numpy as np
import time
import pandas as pd

### Initialization

In [2]:
test_path = "../data/training_set/train/"
image_size=128
num_channels=3
num_classes = 3

### Load Data

In [3]:
files = [test_path + f for f in os.listdir(test_path) if f.endswith('.jpg')]
images = []
for fl in files:
    image = cv2.imread(fl)
    image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
    image = image.astype(np.float32)
    image = np.multiply(image, 1.0 / 255.0)
    
    images.append(image)
    
images = np.array(images)

#### notation generating flip & rotation photos

In [ ]:
#flip
image_flip = cv2.flip(image,1)
images.append(image_flip)
add = fl[:21]+'test'+fl[26:-4]+'_flip.jpg'
cv2.imwrite(add, np.multiply(image_flip, 255.0))


#rotate by -15 ~ +15 degree
Left = np.random.uniform(-15, 15)
rows, cols, color = image.shape
M = cv2.getRotationMatrix2D((cols/2, rows/2), Left, 1)
image_rotate = cv2.warpAffine(image, M, (cols, rows))
images.append(image_rotate)
add = fl[:21]+'test'+fl[26:-4]+'_rotate.jpg'
cv2.imwrite(add, np.multiply(image_rotate, 255.0))

### Testing on given dataset

label (0 for muffin, 1 for chicken, 2 for dog)

In [7]:
saver = tf.train.import_meta_graph('where_are_my_puppies.meta')
sess = tf.Session()
saver.restore(sess, "./where_are_my_puppies")
graph = tf.get_default_graph()

y_pred = graph.get_tensor_by_name("y_pred:0")
y_pred_cls = tf.argmax(y_pred, axis=1)
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
layer_fc2 = graph.get_tensor_by_name("layer_fc1:0")

results = []
for image in images:
    x_batch = image.reshape(1, image_size,image_size,num_channels)
    y_test_images = np.zeros((1, num_classes)) 
    feed_dict_testing = {x: x_batch, y_true: y_test_images}
    result=sess.run(layer_fc2, feed_dict=feed_dict_testing)
    results.append(result)
    

INFO:tensorflow:Restoring parameters from ./where_are_my_puppies


In [16]:
df = pd.DataFrame(np.resize(np.asarray(results), (len(files),len(results[0][0]))))
df.to_csv("../output/feature_CNN_test.csv")